In [11]:
import pandas as pd
import numpy as np
import re

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

--------------
## Problems that were discovered later on:
we had to come back here to take care of them

### <font color='red'>Problem1:</font> Incomplete data
see: [4_EDA](./4_EDA.ipynb)

## At the EDA stage we discovered that the data collection was not carried out properly:


in the 'game_crawling' file we saved the missing games data to 'mid_rated_games.csv' file

see: [game_crawling](./2_game_crawling.ipynb) => <font color='red'>'Problem1: Incomplete data'</font>

## Now we will add the file to the list of files to be handled

In [12]:
file_name3 = "games_data/mid_rated_games.csv"

--------------

### Combining the data files into one data frame

In [13]:
file_name1 = "games_data/lowest_rated_games.csv"
file_name2 = "games_data/top_rated_games.csv"
## Added file - from problem1 ##
file_name3 = "games_data/mid_rated_games.csv"
df = pd.concat(
    map(pd.read_csv, [file_name1, file_name2, file_name3]), ignore_index=True)

Deletion of the index column - "Unnamed: 0"

In [14]:
df.drop("Unnamed: 0", axis=1, inplace=True)

--------------
Checking the data

In [15]:
df.head()

,game_name,platform,publisher,release_date,meta_scroe,user_score,develeoper,genres,num_of_players,rating,user_positive_review,user_negative_review,user_mixed_review,critic_positive_review,critic_negative_review,critic_mixed_review
0,Tharsis,PC,Choice Provisions,"Jan 11, 2016",61.0,4.8,Choice Provisions,"Miscellaneous, Board / Card Game",No Online Multiplayer,M,19,27,11.0,3.0,3.0,17.0
1,WWI Tannenberg: Eastern Front,PlayStation 4,Blackmill Games,"Jul 24, 2020",61.0,8.3,M2H,"Action, Shooter, First-Person, Tactical",Up to 40,M,4,0,2.0,3.0,1.0,6.0
2,Dungeons 2,PlayStation 4,Kalypso,"May 24, 2016",61.0,6.6,Kalypso,"Strategy, General",Up to 4,T,15,6,4.0,1.0,1.0,7.0
3,WE ARE DOOMED,PlayStation 4,Vertex Pop,"Apr 14, 2015",61.0,5.9,Vertex Pop,"Action, Shooter, Shoot-'Em-Up, Horizontal",NaN,E,10,9,3.0,2.0,1.0,6.0
4,Sixty Second Shooter Prime,Xbox One,Happion Laboratories,"Jun 18, 2014",61.0,7.1,Happion Laboratories,"Action, Shooter, Static, Shoot-'Em-Up, Top-Down",NaN,E,9,2,5.0,0.0,1.0,7.0


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15090 entries, 0 to 15089
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_name               15090 non-null  object 
 1   platform                15090 non-null  object 
 2   publisher               15082 non-null  object 
 3   release_date            15090 non-null  object 
 4   meta_scroe              15083 non-null  float64
 5   user_score              15087 non-null  object 
 6   develeoper              15073 non-null  object 
 7   genres                  15090 non-null  object 
 8   num_of_players          12157 non-null  object 
 9   rating                  13550 non-null  object 
 10  user_positive_review    14108 non-null  object 
 11  user_negative_review    14108 non-null  object 
 12  user_mixed_review       14108 non-null  object 
 13  critic_positive_review  15085 non-null  float64
 14  critic_negative_review  15085 non-null

--------------

Dropping the nan values for: 'meta_scroe', 'publisher', 'develeoper':

In [18]:
df.dropna(subset=['meta_scroe'], inplace=True)
df.dropna(subset=['publisher'], inplace=True)
df.dropna(subset=['develeoper'], inplace=True)

## In user_score there are some 'tbd' values 
'tbd' = to be declade

In [19]:
df[df['user_score'] == "tbd"].head()

,game_name,platform,publisher,release_date,meta_scroe,user_score,develeoper,genres,num_of_players,rating,user_positive_review,user_negative_review,user_mixed_review,critic_positive_review,critic_negative_review,critic_mixed_review
25,King Arthur,Xbox,Konami,"Nov 16, 2004",61.0,tbd,Krome Studios,"Action Adventure, Fantasy",1-2,T,0,1,1.0,2.0,2.0,11.0
30,The Messenger (2001),PC,DreamCatcher Interactive,"Feb 14, 2001",61.0,tbd,Index Multimedia,"Adventure, General, General",1 Player,T,NaN,NaN,NaN,2.0,1.0,11.0
32,Hunter's Legacy,Xbox One,Lienzo,"Jan 20, 2017",61.0,tbd,Lienzo,"Action, Action Adventure, Platformer, Open-Wor...",No Online Multiplayer,E,0,0,1.0,1.0,1.0,5.0
38,Nikoli's Pencil Puzzle,3DS,Konami,"Oct 25, 2011",61.0,tbd,NATSUME ATARI Inc.,"Miscellaneous, Puzzle, Puzzle, Puzzle, Logic, ...",NaN,E,2,1,0.0,1.0,0.0,6.0
41,Maglam Lord,PlayStation 4,PQube,"Feb 4, 2022",61.0,tbd,Felistella,"Role-Playing, Action RPG",No Online Multiplayer,T,NaN,NaN,NaN,0.0,0.0,7.0


changing the 'tbd' to nan values:

In [20]:
df.replace("tbd", np.nan, inplace=True)
df[df["user_score"] == "tbd"]

,game_name,platform,publisher,release_date,meta_scroe,user_score,develeoper,genres,num_of_players,rating,user_positive_review,user_negative_review,user_mixed_review,critic_positive_review,critic_negative_review,critic_mixed_review


## Filling the user_score nan values with the mean values

In [22]:
df["user_score"] = df["user_score"].astype("float64")
df["user_score"].fillna(df["user_score"].mean(), inplace=True)

--------------
Checking the data

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15059 entries, 0 to 15089
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_name               15059 non-null  object 
 1   platform                15059 non-null  object 
 2   publisher               15059 non-null  object 
 3   release_date            15059 non-null  object 
 4   meta_scroe              15059 non-null  float64
 5   user_score              15059 non-null  float64
 6   develeoper              15059 non-null  object 
 7   genres                  15059 non-null  object 
 8   num_of_players          12138 non-null  object 
 9   rating                  13533 non-null  object 
 10  user_positive_review    14086 non-null  object 
 11  user_negative_review    14086 non-null  object 
 12  user_mixed_review       14086 non-null  object 
 13  critic_positive_review  15059 non-null  float64
 14  critic_negative_review  15059 non-null

--------------

### Data with large numbers are with commas

In [26]:
df["user_positive_review"][6000]

'1,293'

Therefore, a simple conversion to the int type will not help here

# Review Data handling

## Conversion of all review data

In [27]:
colums = ["user_positive_review", "user_negative_review", "user_mixed_review"]
for column in colums:
    nums = []
    for string in df[column]:
        if(type(string) != float):
            nums.append(int(string.replace(",", "")))
        else:
            nums.append(np.nan)
    df[column] = nums

## Filling the review data nan values with the mean values

In [28]:
columns = [
    "user_positive_review", "user_negative_review", "user_mixed_review",
    "critic_positive_review", "critic_negative_review", "critic_mixed_review"
]
for column in columns:
    df[column] = df[column].astype("float")
    df[column].fillna(df[column].mean(), inplace=True)

## Normalizing the review data

In [29]:
# making a new normalize column of 'user_positive_review'
user_review = df.iloc[:,10:13].copy()
user_review_sum = user_review["user_positive_review"] + user_review["user_negative_review"] + user_review["user_mixed_review"]
user_review["user_positive_normalize"] = user_review["user_positive_review"] / user_review_sum

df["user_positive_normelize"] = user_review["user_positive_normalize"]


In [30]:
# making a new normalize column of 'critic_positive_review'
critic_review = df.iloc[:,-4:-1].copy()
critic_review_sum = critic_review["critic_positive_review"] + critic_review["critic_mixed_review"] + critic_review["critic_negative_review"]
critic_review["critic_positive_normalize"] = critic_review["critic_positive_review"] / critic_review_sum

df["critic_positive_normelize"] = critic_review["critic_positive_normalize"]


In [31]:
df.drop(columns, axis=1, inplace=True)

--------------
Checking the data

In [32]:
df.head()

,game_name,platform,publisher,release_date,meta_scroe,user_score,develeoper,genres,num_of_players,rating,user_positive_normelize,critic_positive_normelize
0,Tharsis,PC,Choice Provisions,"Jan 11, 2016",61.0,4.8,Choice Provisions,"Miscellaneous, Board / Card Game",No Online Multiplayer,M,0.234607,0.130435
1,WWI Tannenberg: Eastern Front,PlayStation 4,Blackmill Games,"Jul 24, 2020",61.0,8.3,M2H,"Action, Shooter, First-Person, Tactical",Up to 40,M,0.102600,0.300000
2,Dungeons 2,PlayStation 4,Kalypso,"May 24, 2016",61.0,6.6,Kalypso,"Strategy, General",Up to 4,T,0.267923,0.111111
3,WE ARE DOOMED,PlayStation 4,Vertex Pop,"Apr 14, 2015",61.0,5.9,Vertex Pop,"Action, Shooter, Shoot-'Em-Up, Horizontal",NaN,E,0.185232,0.222222
4,Sixty Second Shooter Prime,Xbox One,Happion Laboratories,"Jun 18, 2014",61.0,7.1,Happion Laboratories,"Action, Shooter, Static, Shoot-'Em-Up, Top-Down",NaN,E,0.195710,0.000000


--------------

# Adding 'max_players' and 'online_game' columns
With the information in the 'num_of_players' column

In [33]:
num_of_players_unique = list(df["num_of_players"].unique())

replace_map_for_max_players = {}
replace_map_for_online_game = {}

for val in num_of_players_unique:
    if(type(val) != float):
        max_player = np.nan
        max_player_list = [int(s) for s in re.findall(r'\b\d+\b', val)]
        if(max_player_list):
            max_player = max(max_player_list)
            replace_map_for_max_players[val] = max_player
        else:
            replace_map_for_max_players[val] = 1

        
        online = ('Online' in val) and ('No Online' not in val)
        if(max_player != np.nan):
            if(max_player >= 4):
                online = True
        
        if(online):
            replace_map_for_online_game[val] = 1
        else:
            replace_map_for_online_game[val] = 0
            
    else:
        replace_map_for_max_players[val] = 1
        replace_map_for_online_game[val] = 0
            
df["max_players"] = df["num_of_players"].copy()
df["online_game"] = df["num_of_players"].copy()

df["max_players"].replace(replace_map_for_max_players, inplace=True)
df["online_game"].replace(replace_map_for_online_game, inplace=True)

df.drop("num_of_players", axis=1, inplace=True)

### 'online_game' column:

1 = online

0 = not online

In [34]:
df["online_game"].unique()

array([0, 1], dtype=int64)

### 'max_players' column:

maximum players possible in a game

In [35]:
df["max_players"].unique()

array([ 1, 40,  4,  2,  3, 12,  8,  6,  5, 10, 16, 64, 32, 44,  9, 24, 20,
       60, 30, 14, 22, 18, 15, 36], dtype=int64)

In [36]:
# df = pd.get_dummies(df, columns=["online_game"], prefix=["online_game"])

--------------
Checking the data

In [37]:
df.head()

,game_name,platform,publisher,release_date,meta_scroe,user_score,develeoper,genres,rating,user_positive_normelize,critic_positive_normelize,max_players,online_game
0,Tharsis,PC,Choice Provisions,"Jan 11, 2016",61.0,4.8,Choice Provisions,"Miscellaneous, Board / Card Game",M,0.234607,0.130435,1,0
1,WWI Tannenberg: Eastern Front,PlayStation 4,Blackmill Games,"Jul 24, 2020",61.0,8.3,M2H,"Action, Shooter, First-Person, Tactical",M,0.102600,0.300000,40,1
2,Dungeons 2,PlayStation 4,Kalypso,"May 24, 2016",61.0,6.6,Kalypso,"Strategy, General",T,0.267923,0.111111,4,1
3,WE ARE DOOMED,PlayStation 4,Vertex Pop,"Apr 14, 2015",61.0,5.9,Vertex Pop,"Action, Shooter, Shoot-'Em-Up, Horizontal",E,0.185232,0.222222,1,0
4,Sixty Second Shooter Prime,Xbox One,Happion Laboratories,"Jun 18, 2014",61.0,7.1,Happion Laboratories,"Action, Shooter, Static, Shoot-'Em-Up, Top-Down",E,0.195710,0.000000,1,0


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15059 entries, 0 to 15089
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   game_name                  15059 non-null  object 
 1   platform                   15059 non-null  object 
 2   publisher                  15059 non-null  object 
 3   release_date               15059 non-null  object 
 4   meta_scroe                 15059 non-null  float64
 5   user_score                 15059 non-null  float64
 6   develeoper                 15059 non-null  object 
 7   genres                     15059 non-null  object 
 8   rating                     13533 non-null  object 
 9   user_positive_normelize    15059 non-null  float64
 10  critic_positive_normelize  15059 non-null  float64
 11  max_players                15059 non-null  int64  
 12  online_game                15059 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usag

--------------

## We will change the names of the platforms to make them more general

In [39]:
df["platform"].unique()

array(['PC', 'PlayStation 4', 'Xbox One', 'PSP', '3DS', 'Switch',
       'Xbox 360', 'PlayStation 2', 'PlayStation 3', 'DS', 'Xbox',
       'Game Boy Advance', 'PlayStation', 'PlayStation Vita', 'Dreamcast',
       'PlayStation 5', 'GameCube', 'Wii U', 'Xbox Series X', 'Wii',
       'Nintendo 64'], dtype=object)

In [40]:
platform_replace_map = {
    'PSP': 'PlayStation',
    '3DS': 'Nintendo',
    'Switch': 'Nintendo',
    'DS': 'Nintendo',
    'Game Boy Advance': 'Nintendo',
    'Dreamcast': 'Old Platform',
    'GameCube': 'Old Platform',
    'Nintendo 64': 'Nintendo',
    'Stadia': 'PC'
}
platform_unique_list = list(df["platform"].unique())
for val in platform_unique_list:
    replaceXbox = re.findall(r'(Xbox) ', val)
    replacePlayStation = re.findall(r'(PlayStation) ', val)
    replaceWii = re.findall(r'(Wii) ', val)
    if(replaceXbox):
        platform_replace_map[val] = replaceXbox[0]
    if(replacePlayStation):
        platform_replace_map[val] = replacePlayStation[0]
    if(replaceWii):
        platform_replace_map[val] = replaceWii[0]

df["platform"].replace(platform_replace_map, inplace=True)

In [41]:
df["platform"].unique()

array(['PC', 'PlayStation', 'Xbox', 'Nintendo', 'Old Platform', 'Wii'],
      dtype=object)

## We will assign to each rating a value related to the order of the rating

In [42]:
df["rating"].unique()

array(['M', 'T', 'E', 'E10+', nan, 'RP', 'AO', 'K-A'], dtype=object)

first we replace the nan valuse with the top value:

In [43]:
top_val = df["rating"].describe()[2]
df["rating"].fillna(top_val, inplace=True)

The values will be distributed like this:
- RP - Rating Pending == 0
- E - Everyone 6 and older == 1
- E10+ - 10 and older == 2
- K-A - 10 and older == 2
- T - Teen 13 and older == 3
- m - 17+ == 4
- AO - Adults Only 18 years and older == 5

In [44]:
rating_replace_map = {
    'RP': 0, 'E': 1, 'E10+': 2,
    'K-A': 2, 'T': 3, 'M': 4, 'AO': 5}

df["rating"].replace(rating_replace_map, inplace=True)

## scaling the 'meta_scroe' and 'user_score' columns

In [45]:
min_max_scaler = preprocessing.MinMaxScaler()
meta_scroe_scaled = min_max_scaler.fit_transform(df[['meta_scroe']])
user_score_scaled = min_max_scaler.fit_transform(df[['user_score']])
df['meta_score'] = meta_scroe_scaled
df['user_score'] = user_score_scaled
df.drop("meta_scroe", axis=1, inplace=True)

## The 'release_date' will be divided to two columns: 'release_month' and 'release_year'

In [46]:
df["release_date"].unique()

array(['Jan 11, 2016', 'Jul 24, 2020', 'May 24, 2016', ...,
       'Aug 19, 2003', 'Feb 27, 2020', 'Jun 10, 2019'], dtype=object)

In [47]:
import calendar

In [48]:
# Getting months by their numbers
month_number = {}
for index, month in enumerate(calendar.month_abbr):
    if(month):
        month_number[month.lower()] = index

# Divides into two columns:
release_date_list = list(df["release_date"])
release_year = []
release_month = []
for date in release_date_list:
    year = re.findall(r'\b\d+\b', date)[1]
    release_year.append(int(year))
    month = re.findall(r'(\w\w\w) ', date)[0]
    release_month.append(month_number[month.lower()])

df['release_year'] = release_year
df['release_month'] = release_month

In [49]:
df.drop("release_date", axis=1, inplace=True)

--------------
Checking the data

In [50]:
df.head()

,game_name,platform,publisher,user_score,develeoper,genres,rating,user_positive_normelize,critic_positive_normelize,max_players,online_game,meta_score,release_year,release_month
0,Tharsis,PC,Choice Provisions,0.478723,Choice Provisions,"Miscellaneous, Board / Card Game",4,0.234607,0.130435,1,0,0.568182,2016,1
1,WWI Tannenberg: Eastern Front,PlayStation,Blackmill Games,0.851064,M2H,"Action, Shooter, First-Person, Tactical",4,0.102600,0.300000,40,1,0.568182,2020,7
2,Dungeons 2,PlayStation,Kalypso,0.670213,Kalypso,"Strategy, General",3,0.267923,0.111111,4,1,0.568182,2016,5
3,WE ARE DOOMED,PlayStation,Vertex Pop,0.595745,Vertex Pop,"Action, Shooter, Shoot-'Em-Up, Horizontal",1,0.185232,0.222222,1,0,0.568182,2015,4
4,Sixty Second Shooter Prime,Xbox,Happion Laboratories,0.723404,Happion Laboratories,"Action, Shooter, Static, Shoot-'Em-Up, Top-Down",1,0.195710,0.000000,1,0,0.568182,2014,6


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15059 entries, 0 to 15089
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   game_name                  15059 non-null  object 
 1   platform                   15059 non-null  object 
 2   publisher                  15059 non-null  object 
 3   user_score                 15059 non-null  float64
 4   develeoper                 15059 non-null  object 
 5   genres                     15059 non-null  object 
 6   rating                     15059 non-null  int64  
 7   user_positive_normelize    15059 non-null  float64
 8   critic_positive_normelize  15059 non-null  float64
 9   max_players                15059 non-null  int64  
 10  online_game                15059 non-null  int64  
 11  meta_score                 15059 non-null  float64
 12  release_year               15059 non-null  int64  
 13  release_month              15059 non-null  int

--------------

## The genres are listed in long lists
# Extracting the main genres:

In [53]:
df["genres"]

0                         Miscellaneous, Board / Card Game
1                  Action, Shooter, First-Person, Tactical
2                                        Strategy, General
3                Action, Shooter, Shoot-'Em-Up, Horizontal
4          Action, Shooter, Static, Shoot-'Em-Up, Top-Down
                               ...                        
15085                               Action, Platformer, 2D
15086                       Racing, Simulation, Automobile
15087    Miscellaneous, Puzzle, Action, Puzzle, Puzzle,...
15088                         Action Adventure, Open-World
15089                         Driving, Racing, GT / Street
Name: genres, Length: 15059, dtype: object

Extracting the names of the genres

In [55]:
genres_names = set()
for genre in df["genres"]:
    genre_list = genre.split(", ")
    for name in genre_list:
        if(name):
            genres_names.add(name)
len(genres_names)

169

### To check which genres are the main ones:

- we will count each one

- and we will define the main genres as the genres with the most iterations in the data frame

In [56]:
genres_count = {'': 0}
for genre in genres_names:
    counter = 0
    for val in df["genres"]:
        if(genre in val):
            counter = counter + 1
    genres_count[genre] = counter
len(genres_count)

170

In [57]:
main_genres_per_game = []
for game_genres in df["genres"]:
    genre_list = game_genres.split(", ")
    max_genre = max(genre_list, key=lambda val: genres_count[val])
    main_genres_per_game.append(max_genre)
df["main_genre"] = main_genres_per_game

In [58]:
df.drop("genres", axis=1, inplace=True)

----------------
#### Reordering the dataframe

In [59]:
columns_order = ["game_name", "platform", "main_genre",
                 "develeoper", "publisher", "max_players",
                 "online_game", "release_year", "release_month",
                 "rating", "meta_score", "user_score",
                 "critic_positive_normelize", "user_positive_normelize",
                ]
df = df[columns_order]
df.head()

,game_name,platform,main_genre,develeoper,publisher,max_players,online_game,release_year,release_month,rating,meta_score,user_score,critic_positive_normelize,user_positive_normelize
0,Tharsis,PC,Miscellaneous,Choice Provisions,Choice Provisions,1,0,2016,1,4,0.568182,0.478723,0.130435,0.234607
1,WWI Tannenberg: Eastern Front,PlayStation,Action,M2H,Blackmill Games,40,1,2020,7,4,0.568182,0.851064,0.300000,0.102600
2,Dungeons 2,PlayStation,General,Kalypso,Kalypso,4,1,2016,5,3,0.568182,0.670213,0.111111,0.267923
3,WE ARE DOOMED,PlayStation,Action,Vertex Pop,Vertex Pop,1,0,2015,4,1,0.568182,0.595745,0.222222,0.185232
4,Sixty Second Shooter Prime,Xbox,Action,Happion Laboratories,Happion Laboratories,1,0,2014,6,1,0.568182,0.723404,0.000000,0.195710


----------------

categorising develeoper and publisher columns:

In [61]:
df["develeoper"] = df["develeoper"].astype('category')
df["publisher"] = df["publisher"].astype('category')

----------------

## we have meny duplicates in the data

In [62]:
df[df["game_name"] == "Assassin's Creed Origins"]

,game_name,platform,main_genre,develeoper,publisher,max_players,online_game,release_year,release_month,rating,meta_score,user_score,critic_positive_normelize,user_positive_normelize
5769,Assassin's Creed Origins,Xbox,Action Adventure,Ubisoft,Ubisoft,1,0,2017,10,4,0.840909,0.787234,0.880952,0.743562
6447,Assassin's Creed Origins,PC,Action Adventure,Ubisoft,Ubisoft,1,0,2017,10,4,0.829545,0.702128,0.896552,0.584393
7995,Assassin's Creed Origins,PlayStation,Action Adventure,Ubisoft,Ubisoft,1,0,2017,10,4,0.795455,0.744681,0.841270,0.641498


## Handling duplicates

In [63]:
duplicates = df[df["game_name"].duplicated(keep=False)].copy()
duplicates_names = list(duplicates["game_name"].unique())
df.reset_index(drop=True, inplace=True)

for duplicate_name in duplicates_names:
    duplicates_game = df[df["game_name"] == duplicate_name]
    
    # saving the mean valu of the duplicates
    meta_score_mean = duplicates_game["meta_score"].mean()
    user_score_mean = duplicates_game["user_score"].mean()
    critic_mean = duplicates_game["critic_positive_normelize"].mean()
    user_mean = duplicates_game["user_positive_normelize"].mean()
    
    row_index = df[df["game_name"] == duplicate_name].index[0]
    df.loc[row_index, "meta_score"] = meta_score_mean
    df.loc[row_index, "user_score"] = user_score_mean
    df.loc[row_index, "critic_positive_normelize"] = critic_mean
    df.loc[row_index, "user_positive_normelize"] = user_mean
    
    # making a platform sting of all the duplicates platforms
    platforms = set()
    for val in duplicates_game["platform"]:
        platforms.add(val)
    # the list will be sorted for future labeling
    platforms = sorted(list(platforms))
    df.loc[row_index, "platform"] = ", ".join(platforms)
    
    # saving the earliest date of the duplicates
    year_month_zip = zip(duplicates_game["release_year"], duplicates_game["release_month"])
    release_year = min(duplicates_game["release_year"])
    release_month = 12
    for year, month in year_month_zip:
        if(year == release_year and month < release_month):
            release_month = month
    df.loc[row_index, "release_year"] = release_year
    df.loc[row_index, "release_month"] = release_month

df.drop_duplicates(subset="game_name", inplace=True)
df.reset_index(drop=True, inplace=True)

In [64]:
len(df["platform"].unique())

49

## making 'exclusive_game' column:
Content:

1 - Games released for only one platform

0 - Otherwise

In [65]:
exclusive_games_list = []
for platforms in df["platform"]:
    platforms_list = platforms.split(", ")
    if(len(platforms_list) != 1 or 'Old Platform' in platforms_list):
        exclusive_games_list.append(0)
    else:
        exclusive_games_list.append(1)
df['exclusive_game'] = exclusive_games_list

After Duplicates Handling

In [66]:
df[df["game_name"] == "Assassin's Creed Origins"]

,game_name,platform,main_genre,develeoper,publisher,max_players,online_game,release_year,release_month,rating,meta_score,user_score,critic_positive_normelize,user_positive_normelize,exclusive_game
4296,Assassin's Creed Origins,"PC, PlayStation, Xbox",Action Adventure,Ubisoft,Ubisoft,1,0,2017,10,4,0.82197,0.744681,0.872925,0.656484,0


---------------

## Labeling the 'publisher', 'develeoper', 'main_genre' and 'platform' columns

In [67]:
df["publisher_labeled"] = LabelEncoder().fit_transform(df["publisher"])
df["develeoper_labeled"] = LabelEncoder().fit_transform(df["develeoper"])
df["main_genres_labeled"] = LabelEncoder().fit_transform(df["main_genre"])
df["platform_labeled"] = LabelEncoder().fit_transform(df["platform"])

# Final result

In [68]:
df.head()

,game_name,platform,main_genre,develeoper,publisher,max_players,online_game,release_year,release_month,rating,meta_score,user_score,critic_positive_normelize,user_positive_normelize,exclusive_game,publisher_labeled,develeoper_labeled,main_genres_labeled,platform_labeled
0,Tharsis,"Nintendo, PC, PlayStation",Miscellaneous,Choice Provisions,Choice Provisions,1,0,2016,1,4,0.571970,0.607182,0.230978,0.464190,0,266,551,6,9
1,WWI Tannenberg: Eastern Front,PlayStation,Action,M2H,Blackmill Games,40,1,2020,7,4,0.568182,0.851064,0.300000,0.102600,1,177,1684,0,42
2,Dungeons 2,PlayStation,General,Kalypso,Kalypso,4,1,2016,5,3,0.568182,0.670213,0.111111,0.267923,1,772,1525,5,42
3,WE ARE DOOMED,PlayStation,Action,Vertex Pop,Vertex Pop,1,0,2015,4,1,0.568182,0.595745,0.222222,0.185232,1,1552,3188,0,42
4,Sixty Second Shooter Prime,Xbox,Action,Happion Laboratories,Happion Laboratories,1,0,2014,6,1,0.568182,0.723404,0.000000,0.195710,1,649,1260,0,48


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10569 entries, 0 to 10568
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   game_name                  10569 non-null  object  
 1   platform                   10569 non-null  object  
 2   main_genre                 10569 non-null  object  
 3   develeoper                 10569 non-null  category
 4   publisher                  10569 non-null  category
 5   max_players                10569 non-null  int64   
 6   online_game                10569 non-null  int64   
 7   release_year               10569 non-null  int64   
 8   release_month              10569 non-null  int64   
 9   rating                     10569 non-null  int64   
 10  meta_score                 10569 non-null  float64 
 11  user_score                 10569 non-null  float64 
 12  critic_positive_normelize  10569 non-null  float64 
 13  user_positive_normelize    1056

## Saving data in 'all_games.csv' file

In [70]:
# df.to_csv(f'games_data/all_games.csv')

## Saving data in 'all_games2.csv' file
### Added after <font color='red'>Problem1:</font> Incomplete data

In [71]:
df.to_csv(f'games_data/all_games2.csv')

-------------------